In [ ]:
import psycopg2
import os
from dotenv import load_dotenv


load_dotenv()


conn = psycopg2.connect(
    host=os.getenv("POSTGRES_HOST"),
    port=os.getenv("POSTGRES_PORT"),
    user=os.getenv("POSTGRES_USER"),
    password=os.getenv("POSTGRES_PASSWORD"),
    dbname=os.getenv("POSTGRES_DB")
)


In [5]:
cursor = conn.cursor()

In [10]:
cursor.execute("SELECT version();")
print(cursor.fetchone())

('PostgreSQL 14.17 on x86_64-pc-linux-musl, compiled by gcc (Alpine 14.2.0) 14.2.0, 64-bit',)


In [36]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS brasileirao_data (
    id SERIAL PRIMARY KEY,
    placement INT NOT NULL,
    name VARCHAR(255) NOT NULL,
    points INT NOT NULL,
    total_games INT NOT NULL,
    wins INT NOT NULL,
    draws INT NOT NULL,
    loses INT NOT NULL,
    goals_scored INT NOT NULL,
    goals_conceded INT NOT NULL,
    goals_difference INT NOT NULL,
    season INT NOT NULL,
    winning_percentage NUMERIC(5,2) NOT NULL
)
""")
conn.commit()

In [37]:
import pandas as pd


transformed_data_path = "../data/transformed.csv"
df = pd.read_csv(transformed_data_path)
df.head()

,placement,name,points,total_games,wins,draws,loses,goals_scored,goals_conceded,goals_difference,season,winning_percentage
0,1,Cruzeiro,100,46,31,7,8,102,47,55,2003,72.46
1,2,Santos,87,46,25,12,9,93,60,33,2003,63.04
2,3,São Paulo,78,46,22,12,12,81,67,14,2003,56.52
3,4,São Caetano,74,46,19,14,13,53,37,16,2003,53.62
4,5,Coritiba,73,46,21,10,15,67,58,9,2003,52.90


In [38]:
clubs_list = [tuple(row) for i, row in df.iterrows()]
clubs_list[:3]

[(1, 'Cruzeiro', 100, 46, 31, 7, 8, 102, 47, 55, 2003, 72.46),
 (2, 'Santos', 87, 46, 25, 12, 9, 93, 60, 33, 2003, 63.04),
 (3, 'São Paulo', 78, 46, 22, 12, 12, 81, 67, 14, 2003, 56.52)]

In [39]:
query = "INSERT INTO brasileirao_data (placement, name, points, total_games, wins, draws, loses, goals_scored, goals_conceded, goals_difference, season, winning_percentage) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

In [40]:
cursor.executemany(query, clubs_list)
conn.commit()

In [41]:
print(cursor.rowcount, "rows inserted")

430 rows inserted


In [42]:
df.shape

(430, 12)

In [43]:
cursor.execute("SELECT * FROM brasileirao_data LIMIT 5")

for row in cursor:
    print(row)

(1, 1, 'Cruzeiro', 100, 46, 31, 7, 8, 102, 47, 55, 2003, Decimal('72.46'))
(2, 2, 'Santos', 87, 46, 25, 12, 9, 93, 60, 33, 2003, Decimal('63.04'))
(3, 3, 'São Paulo', 78, 46, 22, 12, 12, 81, 67, 14, 2003, Decimal('56.52'))
(4, 4, 'São Caetano', 74, 46, 19, 14, 13, 53, 37, 16, 2003, Decimal('53.62'))
(5, 5, 'Coritiba', 73, 46, 21, 10, 15, 67, 58, 9, 2003, Decimal('52.90'))


In [44]:
conn.close()